TODO:
Divide into strips w/ grand staff
Calculate median pixel value in each row (close to 0, then very high etc)
Use this row median graph, cut into indivdiual staffs, then go into coord system by mapping vertical position to one note (e.g. right goes C4 to B6),
left goes from C2 to B3

How can we est the location of the 5 peaks + do linear interp? Lowest peak = E4, then G4, then B4, D5, F5.

Touch base with Mint so she can point to appropriate code and you can try using it after fixing the graph  to log. 

Use code Mint points you to in order to find a way to convert each score into individual staff lines (e.g. just RH, just LH), calculate median of each row, est location of 5 peaks, then do linear interpolation w/ built-in SKLearn/Numpy to d o a linear interp of the 5 

Give  it pairs of x,y coords. So like notes 0,2,4,6,8, ... up to 20 so 21-dim vector. y-axis is which note out of 21, then x-axis is the vertical pixel position from the bottom (e.g. 35 pixels until then)--use lin interp to convert from vertical pixel position to discretized note value. Then can do a look up in a hash table or smth. 

1. Pull out individual strips from a page of sheet music
2. Mint's intial code for above things--try to est conversion from vertical pixel position to discretized note value. 

In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from scipy import ndimage
from scipy import signal
from scipy.stats import linregress
from scipy.signal import convolve2d, gaussian, argrelextrema
from scipy.interpolate import interp1d
from sklearn.preprocessing import normalize
from skimage.transform import rotate
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pickle as cPickle
import librosa
import glob
import re
import sys
import os
import mido
from mido import MidiFile
import cv2
from PIL import Image

In [3]:
def sort_nicely(l):
    d = {}
    for elem in l:
        m = re.search('\S+-(\d+)\.pkl', elem)
        if m:
            idx = int(m.group(1))
            d[idx] = elem
        else:
            print('Failed regular expression matching: %s' % (elem))
            sys.exit(1)
    sorted_filenames = [d[key] for key in sorted(d.keys())]
    return sorted_filenames

In [4]:
def loadStripData(basename):
    pkl_files = sort_nicely(glob.glob(pkl_dir + '/' + basename + '-*.pkl'))
    strips = []
    for pkl_file in pkl_files:
        with open(pkl_file, 'rb') as f:
            d = cPickle.load(f)
            strips.extend(d['strips'])
    return strips

In [5]:
def showImage(X, sz = (12,12)):
    plt.figure(figsize=sz)
    plt.imshow(1-X, cmap='gray')

In [6]:
def showStrips(strips):
    for strip in strips:
        showImage(strip)

In [7]:
pkl_dir = 'line_pred_pkl_data'

In [8]:
scoreFile = 'kv398_vfinale-0'

In [9]:
strips = loadStripData(scoreFile)

In [11]:
len(strips)

0

In [10]:
showImage(strips[0])

IndexError: list index out of range